In [49]:
import pandas as pd
import numpy as np
import requests
import collections
import pickle
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from scipy import spatial
import networkx as nx
import plotly.graph_objects as go

In [2]:
cd /content/drive/MyDrive/Data Analyst/Spotify

/content/drive/MyDrive/Data Analyst/Spotify


In [3]:
df = pd.read_csv('./indie_df.csv')
df.head()

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,...,analysis_url,duration_ms,time_signature,name,song_artist_names,artist_name,song_popularity,released_date,duration_m,published_days
0,0.650,0.195,5,-13.678,1,0.0495,0.6450,0.000797,0.107,0.375,...,https://api.spotify.com/v1/audio-analysis/6SiV...,250485,4,Da Lau Roi - Cheung,['Cheung'],Cheung,34,2021-02-25,4.17,494
1,0.665,0.277,0,-12.457,0,0.0336,0.8690,0.000239,0.158,0.236,...,https://api.spotify.com/v1/audio-analysis/0AFG...,250539,3,Hoàng Hôn - Cheung,['Cheung'],Cheung,46,2019-05-10,4.18,1151
2,0.896,0.668,9,-9.557,1,0.1310,0.0135,0.000151,0.163,0.137,...,https://api.spotify.com/v1/audio-analysis/7juB...,119562,4,Cheung Beat - Cheung,"['Socr8s', 'Cheung']",Cheung,1,2022-05-01,1.99,64
3,0.717,0.470,0,-9.917,1,0.0350,0.4790,0.026200,0.106,0.675,...,https://api.spotify.com/v1/audio-analysis/4YNO...,199000,4,Leilah - Mademoiselle,['Mademoiselle'],Mademoiselle,20,2022-01-20,3.32,165
4,0.755,0.454,7,-10.255,1,0.0703,0.0783,0.596000,0.206,0.540,...,https://api.spotify.com/v1/audio-analysis/5YBU...,201000,4,When we - Mademoiselle,['Mademoiselle'],Mademoiselle,15,2022-01-20,3.35,165


In [ ]:
# Standardize scale features
standardized_feature = StandardScaler().fit_transform(df[['danceability', 'energy', 'loudness', 'mode', 'speechiness',
                                                          'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo']])
standardized_feature

In [6]:
# Find the song that is nearest to the average point
def find_most_average_song(standardized_feature):
    min_dist = 99999999
    min_index = 0
    for i in range(len(standardized_feature)):
        dist = sum([num**2 for num in standardized_feature[i]])
        if dist < min_dist:
            min_index = i
            min_dist = dist
    return df.loc[min_index, 'name']

find_most_average_song(standardized_feature)

'Đâu Là Nơi Bình Yên? - WHEE!'

In [ ]:
# Calculate cosine similarity for all pair of songs
similarity = [[] for _ in range(len(standardized_feature))]
for i in range(len(standardized_feature)):
    print(i)
    for j in range(len(standardized_feature)):
        similarity[i].append(1 - spatial.distance.cosine(standardized_feature[i], standardized_feature[j]))

In [ ]:
# Create graph with nodes and edges
G = nx.Graph()

# Add nodes
for i in range(len(standardized_feature)):
    G.add_node(i, size = (df.loc[i, 'song_popularity'] + 5)/3) # Add 5 to display songs with 0 popularity, divide 3 to scale down the bubble in chart

# Add edges
for i in range(len(similarity)):
    print(i)
    for j in range(len(similarity[i])):
        weight = similarity[i][j]
        if weight >= 0.9 and weight != 1: # If cosine distance over 0.9, then they connect
            G.add_edge(i, j, weight=weight)

In [39]:
# Find songs with most connections
dict_song_nodes = {}
for n in G.nodes():
    dict_song_nodes[df.loc[n, 'name']] = G.degree[n]

sorted(dict_song_nodes.items(), key=lambda kv: kv[1], reverse=True)[:20]


[('VÉ ĐI THIÊN ĐƯỜNG (một chiều) - Ngọt', 57),
 ('Chào Em - Minh Cà Ri', 56),
 ('Goldfish - MINH', 50),
 ('Hà Nội Và Em (feat. Thái Đinh) - Thái Đinh', 48),
 ('Hà Nội Và Em (feat. Thái Đinh) - Minh Cà Ri', 48),
 ('Tết Là - Mademoiselle', 47),
 ('Hà Nội Và Em - Thái Đinh', 47),
 ('Hà Nội Và Em - Minh Cà Ri', 47),
 ("Why Don't You Come? - Mademoiselle", 45),
 ('Ba núi ba đèo - Lê Cát Trọng Lý', 45),
 ('ĐÃ TRÔI HẾT SẠCH - Lê Cát Trọng Lý', 44),
 ('Tôi Đem Giấc Mơ Mình - TRANG', 43),
 ('What Is Love? - MINH', 43),
 ('CON MA ĐANG SỐNG - Lê Cát Trọng Lý', 43),
 ('Lại Gần Và Để Anh Ôm Em Lâu Hơn - Acoustic Version - Haisam', 42),
 ('Quả Bóng Màu Hồng - Cá Hồi Hoang', 42),
 ('11h11 - Thái Đinh', 42),
 ('11h11 - NamKun', 42),
 ('Có Những Chiều - Minh Cà Ri', 42),
 ('Là Em (Live Session) - Thái Đinh', 41)]

In [43]:
# Save file
pd.DataFrame(data=sorted(dict_song_nodes.items(), key=lambda kv: kv[1], reverse=True)[:20], 
             columns=['name', 'nodes']).to_csv('/content/song_nodes_df.csv', index=False)

In [37]:
df.iloc[[33, 509, 620]]

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,...,analysis_url,duration_ms,time_signature,name,song_artist_names,artist_name,song_popularity,released_date,duration_m,published_days
33,0.506,0.0917,8,-15.889,1,0.0482,0.962,0.000015,0.1250,0.110,...,https://api.spotify.com/v1/audio-analysis/7rEC...,236850,4,Why Don't You Come? - Mademoiselle,['Mademoiselle'],Mademoiselle,36,2016-10-19,3.95,2084
509,0.446,0.0778,5,-15.198,1,0.0384,0.965,0.000232,0.1180,0.117,...,https://api.spotify.com/v1/audio-analysis/1PSh...,165313,4,VÉ ĐI THIÊN ĐƯỜNG (một chiều) - Ngọt,['Ngọt'],Ngọt,32,2019-06-10,2.76,1120
620,0.488,0.1090,10,-13.551,1,0.0436,0.968,0.006690,0.0951,0.213,...,https://api.spotify.com/v1/audio-analysis/4AfQ...,181944,4,Hà Nội Và Em (feat. Thái Đinh) - Thái Đinh,"['Minh Cà Ri', 'Thái Đinh']",Thái Đinh,14,2019-12-01,3.03,946


In [44]:
!pip install plotly

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [47]:
pos_ = nx.spring_layout(G)

In [50]:
# Function to make edge
def make_edge(x, y, text, width):
    return  go.Scatter(x         = x,
                       y         = y,
                       line      = dict(width = width,
                                   color = 'cornflowerblue'),
                       hoverinfo = 'text',
                       text      = ([text]),
                       mode      = 'lines')
    
# For each edge, make an edge_trace, append to list
edge_trace = []
for edge in G.edges():
    char_1 = edge[0]
    char_2 = edge[1]
    x0, y0 = pos_[char_1]
    x1, y1 = pos_[char_2]
    text   = str(char_1) + '--' + str(char_2) + ': ' + str(G.edges()[edge]['weight'])
            
    trace  = make_edge([x0, x1, None], [y0, y1, None], text, 
                              width = 0.3*G.edges()[edge]['weight']**1.75)
    edge_trace.append(trace)

In [51]:
# Make a node trace
node_trace = go.Scatter(x         = [],
                        y         = [],
                        text      = [],
                        textposition = "top center",
                        textfont_size = 10,
                        mode      = 'markers+text',
                        hoverinfo = 'none',
                        marker    = dict(color = [],
                                         size  = [],
                                         line  = None))
# For each node in midsummer, get the position and size and add to the node_trace
for node in G.nodes():
    x, y = pos_[node]
    node_trace['x'] += tuple([x])
    node_trace['y'] += tuple([y])
    node_trace['marker']['color'] += tuple(['cornflowerblue'])
    node_trace['marker']['size'] += tuple([G.nodes()[node]['size']])
    #node_trace['text'] += tuple(['<b>' + 'o' + '</b>'])

In [52]:
# Customize layout
layout = go.Layout(
    paper_bgcolor='rgba(0,0,0,0)', # transparent background
    plot_bgcolor='rgba(0,0,0,0)', # transparent 2nd background
    xaxis =  {'showgrid': False, 'zeroline': False}, # no gridlines
    yaxis = {'showgrid': False, 'zeroline': False}, # no gridlines
)
# Create figure
fig = go.Figure(layout = layout)
# Add all edge traces
for trace in edge_trace:
    fig.add_trace(trace)
# Add node trace
fig.add_trace(node_trace)
# Remove legend
fig.update_layout(showlegend = False)
# Remove tick labels
fig.update_xaxes(showticklabels = False)
fig.update_yaxes(showticklabels = False)
# Show figure
fig.show()

# Graph of related artists

In [53]:
# Load file
dict_artists_related = pickle.load(open("./dict_artists_related.pkl", "rb"))

In [54]:
# Create graph
G = nx.Graph()

# Add nodes
for artist_name in dict_artists_related:
    G.add_node(artist_name)
    for related_artist_name in dict_artists_related[artist_name]:
        G.add_node(related_artist_name)

# Add edges
for artist_name in dict_artists_related:
    for related_artist_name in dict_artists_related[artist_name]:
        G.add_edge(artist_name, related_artist_name)

In [55]:
# Remove nodes with too few connections
removed_nodes = [node for node in G.nodes() if G.degree(node) <= 2]

for node in removed_nodes:
    G.remove_node(node)

In [57]:
pos_ = nx.spring_layout(G)

In [58]:
# Function to make edge
def make_edge(x, y, text, width):
    return  go.Scatter(x         = x,
                       y         = y,
                       line      = dict(width = width,
                                   color = 'cornflowerblue'),
                       hoverinfo = 'text',
                       text      = ([text]),
                       mode      = 'lines')
    
# For each edge, make an edge_trace, append to list
edge_trace = []
for edge in G.edges():
    char_1 = edge[0]
    char_2 = edge[1]
    x0, y0 = pos_[char_1]
    x1, y1 = pos_[char_2]
    text   = str(char_1) + '--' + str(char_2) 
            
    trace  = make_edge([x0, x1, None], [y0, y1, None], text, width=0.2)
    edge_trace.append(trace)

In [59]:
# Make a node trace
node_trace = go.Scatter(x         = [],
                        y         = [],
                        text      = [],
                        textposition = "top center",
                        textfont_size = 10,
                        mode      = 'markers+text',
                        hoverinfo = 'none',
                        marker    = dict(color = [],
                                         size  = [],
                                         line  = None))
# For each node in midsummer, get the position and size and add to the node_trace
for node in G.nodes():
    x, y = pos_[node]
    node_trace['x'] += tuple([x])
    node_trace['y'] += tuple([y])
    node_trace['marker']['color'] += tuple(['cornflowerblue'])
    node_trace['text'] += tuple(['<b>' + str(node) + '</b>'])

In [60]:
# Customize layout
layout = go.Layout(
    paper_bgcolor='rgba(0,0,0,0)', # transparent background
    plot_bgcolor='rgba(0,0,0,0)', # transparent 2nd background
    xaxis =  {'showgrid': False, 'zeroline': False}, # no gridlines
    yaxis = {'showgrid': False, 'zeroline': False}, # no gridlines
)
# Create figure
fig = go.Figure(layout = layout)
# Add all edge traces
for trace in edge_trace:
    fig.add_trace(trace)
# Add node trace
fig.add_trace(node_trace)
# Remove legend
fig.update_layout(showlegend = False)
# Remove tick labels
fig.update_xaxes(showticklabels = False)
fig.update_yaxes(showticklabels = False)
# Show figure
fig.show()

In [61]:
# Top artists with most connections
dict_artist_nodes = {node: G.degree(node) for node in G.nodes()}
sorted(dict_artist_nodes.items(), key=lambda kv: kv[1], reverse=True)[:20]

[('Vũ Thanh Vân', 41),
 ('Minh Đinh', 39),
 ('The Flob', 39),
 ('Minh Tốc & Lam', 39),
 ('Kiên Trịnh', 38),
 ('Những Đứa Trẻ', 38),
 ('TRANG', 37),
 ('Tùng', 37),
 ('Thành Luke', 37),
 ('The Cassette', 35),
 ('Chủ Tịch Kim', 35),
 ('Haisam', 33),
 ('Cá Hồi Hoang', 31),
 ('Mademoiselle', 30),
 ('Thành Đồng', 30),
 ('Thoại 004', 30),
 ('Mèow Lạc', 29),
 ('Thái Đinh', 26),
 ('Raditori', 25),
 ('7UPPERCUTS', 24)]

In [62]:
# Save file
pd.DataFrame(data=sorted(dict_artist_nodes.items(), key=lambda kv: kv[1], reverse=True)[:20],
             columns=['artist_name', 'connections']).to_csv('/content/artist_nodes_df.csv', index=False)